In [ ]:
import pvlib
import Coords_2_EY
import pandas as pd 
import numpy as np      
import matplotlib.pyplot as plt
import timeit
from solcast_frames.latlng import LatLng
from solcast_frames.radiationframehandler import RadiationFrameHandler
from solcast_frames.powerframehandler import PowerFrameHandler
from datetime import datetime, timedelta, date
import solcast as sc
import requests
from dateutil import tz
import Trip_to_Distinct_Coords
import os
import scipy as sp
import geemap
import json
import openrouteservice
from openrouteservice import convert
from geojson import MultiLineString
from geojson import LineString
from timezonefinder import TimezoneFinder
import pytz
from pytz import timezone

client = openrouteservice.Client(key = '5b3ce3597851110001cf6248d5cc6329856d4104b16de2d55d56f603')

<h2>Import Post-Processed Data</h2>
Data points of distribution centres and stores, and coordinates of predefined trips in NSW, ACT, and VIC.

NSW - Sydney Chilled Distribution Centre to Woolworths Town Hall
VIC - Woolworths Stud Park to Melbourne Chilled Distribution Centre
WA - Perth Chilled Distribution Centre to Woolworths Collie 

In [ ]:
ww_dcs = pd.read_csv("data/ww_dcs_points_ll.csv")
ww_stores = pd.read_csv("data/ww_stores_points_ll.csv")

In [ ]:
VIC_coords = pd.read_csv('data/Trip_Coords/VIC/StudPark_MelbChilled.csv')
VIC_coords.set_index('Unnamed: 0', inplace = True)

In [ ]:
NSW_coords = pd.read_csv('data/Trip_Coords/NSW/SydChilled_TownHall.csv')
NSW_coords.set_index('Unnamed: 0', inplace = True)

In [ ]:
WA_coords = pd.read_csv('data/Trip_Coords/WA/PerthChilled_Collie.csv')
WA_coords.set_index('Unnamed: 0', inplace = True)

<h1>Initial Code Run </h1>
The following code shows how the downloaded PVGIS data is converted into a usuable form for the functions defined in my python scripts of Coords_2_EY. This code only needs to be run when reworking the data/ choosing a different set of coordinates, otherwise continue to load to csv's below.

In [ ]:
VIC_List = os.listdir('data/PVGIS/VIC_trip')
VIC_List = VIC_List[1:]
VIC_List.sort(reverse=True) #VIC TRIP ONLY
VIC_PVGIS_data = {}
offset = 10
for i in range(0,len(VIC_List)):
    df = pd.read_csv('data/PVGIS/VIC_trip/'+ VIC_List[i])
    df.drop(df.tail(12).index,
        inplace = True)
    df['Date'] = pd.to_datetime(df['time'], format = "%Y%m%d:%H%M")
    df['Date']+= pd.Timedelta(hours=offset)
    df['Gb(i)'] = pd.to_numeric(df['Gb(i)'])
    df['Date_dt']=pd.to_datetime(df['Date'])
    df.drop(axis = 1, labels = 'time', inplace = True )
    VIC_PVGIS_data[i]=df

In [ ]:
NSW_List = os.listdir('data/PVGIS/NSW_trip')
NSW_List = NSW_List[1:]
NSW_PVGIS_data = {}
offset = 10
for i in range(0,len(NSW_List)):
    df = pd.read_csv('data/PVGIS/NSW_trip/'+ NSW_List[i])
    df.drop(df.tail(12).index,
        inplace = True)
    df['Date'] = pd.to_datetime(df['time'], format = "%Y%m%d:%H%M")
    df['Gb(i)'] = pd.to_numeric(df['Gb(i)'])
    df['Date']+= pd.Timedelta(hours=offset)
    df['Date_dt']=pd.to_datetime(df['Date'])
    df.drop(axis = 1, labels = 'time', inplace = True )
    NSW_PVGIS_data[i]=df

In [ ]:
WA_List = os.listdir('data/PVGIS/WA_trip')
WA_List = WA_List[1:]
WA_PVGIS_data = {}
offset = 8 
for i in range(0,len(WA_List)):
    df = pd.read_csv('data/PVGIS/WA_trip/'+ WA_List[i])
    df.drop(df.tail(12).index,
        inplace = True)
    df['Date'] = pd.to_datetime(df['time'], format = "%Y%m%d:%H%M")
    df['Date']+= pd.Timedelta(hours=offset)
    df['Date_dt']=pd.to_datetime(df['Date'])
    df['Gb(i)'] = pd.to_numeric(df['Gb(i)'])
    df.drop(axis = 1, labels = 'time', inplace = True )
    WA_PVGIS_data[i]=df

<h1>Importing Previously Processed PVGIS data in CSV format</h1>

In [ ]:
VIC_PVGIS_data = {}
for i in range(0, len(VIC_coords)-1):
    VIC_PVGIS_data[i] = pd.read_csv("data/PVGIS/Processed/VIC_PVGIS_processed_"+f"{i}")

In [ ]:
NSW_PVGIS_data = {}
for i in range(0, len(NSW_coords)-1):
    NSW_PVGIS_data[i] = pd.read_csv("data/PVGIS/Processed/NSW_PVGIS_processed_"+f"{i}")

In [ ]:
WA_PVGIS_data = {}
for i in range(0, len(WA_coords)-1):
    WA_PVGIS_data[i] = pd.read_csv("data/PVGIS/Processed/WA_PVGIS_processed_"+f"{i}")

<h1> Exporting Code for Initial Python Defined Variables </h1>
Import from csv in the following paths for later recall

In [ ]:
for i in range(0,len(NSW_PVGIS_data)):
    NSW_PVGIS_data[i].to_csv('data/PVGIS/Processed/NSW_PVGIS_processed_'+ f"{i}")

In [ ]:
for i in range(0,len(VIC_PVGIS_data)):
    VIC_PVGIS_data[i].to_csv('data/PVGIS/Processed/VIC_PVGIS_processed_'+ f"{i}")

In [ ]:
for i in range(0,len(WA_PVGIS_data)):
    WA_PVGIS_data[i].to_csv('data/PVGIS/Processed/WA_PVGIS_processed_'+ f"{i}")

<h1>Function Definition for Histogram Data</h1>

In [ ]:
def full_stage_downloaded(data,year,month,day,hour,minute,n_panels,P_stc,gamma_p):
    sol_data = Coords_2_EY.data_collation(year,month,day,hour,minute,data)
    processed_sol_data = Coords_2_EY.solar_data_processing(sol_data)
    final = Coords_2_EY.calculate_power_energy(processed_sol_data,n_panels,P_stc,gamma_p)
    return final

In [ ]:
def histogram_data(K,data,hour,minute,n_panels,P_stc,gamma_p,state):
    '''
    Undertakes random analysis of energy yield on a random set of dates between range of available PVGIS data. 
    
    :params K: the number of dates for random analysis, can increase / decrease sample size
    :params data: pre-processed PVGIS data, in a list of data files with each index representative of the order of the coordinates of the trip (see in attached PVGIS_processed folder and import)
    :params hour: start hour of trip of interest
    :params minute: start minutes of trip of interest
    :params n_panels: number of panels being simulated on the moving vehicles
    :params P_stc: Power output of modelled solar panels under standard test conditions, provided by manufacturer
    :params gamma_p: temperature coefficient of power of panels, as a decimal (should be negative)
    :params state: string of the state of interest (must be in Australia)
    
    :returns: a list of energy outputs, the same length as K. 
    '''
    if state ==  'WA':
        test_date1, test_date2 = date(2006, 1, 1), date(2017, 1, 1)
    else:
        test_date1, test_date2 = date(2006, 1, 1), date(2021, 1, 1)
    # getting days between dates
    dates_bet = test_date2 - test_date1
    total_days = dates_bet.days
    # create an array of total days and select K random values without replacement
    randays = np.random.choice(total_days, K, replace=False)
    # getting random dates
    res = [test_date1 + timedelta(days=int(day)) for day in randays]
    
    Date = []
    energy_output = []
    for i in range(0,len(res)):
        year = res[i].year
        month = res[i].month
        day = res[i].day
        Date.append(res[i])
        table = full_stage_downloaded(data,year,month,day,hour,minute,n_panels,P_stc,gamma_p)
        energy_output.append(table['Energy'].sum())
        #print(f'{i}',"/100")
        #print("Current Time =", current_time)
        #print(res[i])
    outputs = pd.DataFrame()
    outputs.insert(0, "Date", Date)
    outputs.insert(1, "Energy Output", energy_output)
    outputs.to_csv('results/randomdates' + '/' + f'{state}' + '/' + f'{state}'+ f'{hour}' + 'nPanels' + f'{n_panels}'  + '.csv')
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Finished ", hour, " ", minute, "Current Time=", current_time)
    return energy_output

In [ ]:
def histogram_plot(dataset): 
    n_obs=len(dataset)
    intervals = round(np.sqrt(n_obs))
    rang = max(dataset) - min(dataset)
    width = rang/intervals 
    bins=[round(min(dataset))+round(width*i) for i in range(0,intervals)]    
    fig, ax = plt.subplots()
    n, bins, patches = ax.hist(dataset, bins)
    ax.set_xlabel('Energy Output (Wh)')
    ax.set_ylabel('Count')
    return fig, ax

<h1> Undertaking Functions on Data by State and Histogram Analysis</h1>

<h2> Obtaining Histogram Data</h2>

Change script here to change the value of K, the number of random dates calculated. The following analysis is modelling Sunpower Maxeon 6 Panels. 

In [ ]:
VIC_morn = histogram_data(1000,VIC_PVGIS_data,7,30,3,400,-0.0029,'VIC')
VIC_lunch = histogram_data(1000,VIC_PVGIS_data,12,30,3,400,-0.0029,'VIC')
VIC_arvo = histogram_data(1000,VIC_PVGIS_data,16,30,3,400,-0.0029,'VIC')

In [ ]:
NSW_morn = histogram_data(1000,NSW_PVGIS_data,7,30,3,400,-0.0029,'NSW')
NSW_lunch = histogram_data(1000,NSW_PVGIS_data,12,30,3,400,-0.0029,'NSW')
NSW_arvo = histogram_data(1000,NSW_PVGIS_data,16,30,3,400,-0.0029,'NSW')

In [ ]:
WA_morn = histogram_data(1000,WA_PVGIS_data,7,36,3,400,-0.0029,'WA')
WA_lunch = histogram_data(1000,WA_PVGIS_data,12,36,3,400,-0.0029,'WA')
WA_arvo = histogram_data(1000,WA_PVGIS_data,16,36,3,400,-0.0029,'WA')

<h2>Import Existing Histogram Data</h2>
If the code has already been executed and you would like to import data for plotting or further analysis.

In [ ]:
VIC_morn = pd.read_csv('results/randomdates/VIC/VIC7nPanels3.csv')
VIC_lunch = pd.read_csv('results/randomdates/VIC/VIC12nPanels3.csv')
VIC_arvo = pd.read_csv('results/randomdates/VIC/VIC16nPanels3.csv')

NSW_morn = pd.read_csv('results/randomdates/NSW/NSW7nPanels3.csv')
NSW_lunch = pd.read_csv('results/randomdates/NSW/NSW12nPanels3.csv')
NSW_arvo = pd.read_csv('results/randomdates/NSW/NSW16nPanels3.csv')

WA_morn = pd.read_csv('results/randomdates/WA/WA7nPanels3.csv')
WA_lunch = pd.read_csv('results/randomdates/WA/WA12nPanels3.csv')
WA_arvo = pd.read_csv('results/randomdates/WA/WA16nPanels3.csv')

<h2>Histogram and Other Plots</h2>

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize = (10,4))
intervals = 20 #n_bins
ax1.set_xlim(0,500)
ax2.set_xlim(0,500)
ax3.set_xlim(0,500)
n_obs1=len(VIC_morn)
rang1 = max(VIC_morn) - min(VIC_morn)
width1 = rang1/intervals
bins1=[round(min(VIC_morn))+round(width1*i) for i in range(0,intervals)]    

n, bins, patches = ax1.hist(VIC_morn, bins1)
ax1.set_xlabel('Energy Output (Wh)')
ax1.set_ylabel('Count')
ax1.set_title('VIC 7:30')

n_obs2=len(VIC_lunch)
rang2 = max(VIC_lunch) - min(VIC_lunch)
width2 = rang2/intervals
bins2=[round(min(VIC_lunch))+round(width2*i) for i in range(0,intervals)]    

n, bins, patches = ax2.hist(VIC_lunch, bins2)
ax2.set_xlabel('Energy Output (Wh)')
ax2.set_title('VIC 12:30')

n_obs3=len(VIC_arvo)
rang3 = max(VIC_arvo) - min(VIC_arvo)
width3 = rang3/intervals
bins3=[round(min(VIC_arvo))+round(width3*i) for i in range(0,intervals)]    

n, bins, patches = ax3.hist(VIC_arvo, bins3)
ax3.set_xlabel('Energy Output (Wh)')
ax3.set_title('VIC 16:30')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize = (10,4))
intervals = 20 #n_bins
ax1.set_xlim(0,500)
ax2.set_xlim(0,500)
ax3.set_xlim(0,500)
n_obs1=len(NSW_morn)
rang1 = max(NSW_morn) - min(NSW_morn)
width1 = rang1/intervals
bins1=[round(min(NSW_morn))+round(width1*i) for i in range(0,intervals)]    

n, bins, patches = ax1.hist(NSW_morn, bins1)
ax1.set_xlabel('Energy Output (Wh)')
ax1.set_ylabel('Count')
ax1.set_title('NSW 7:30')

n_obs2=len(NSW_lunch)
rang2 = max(NSW_lunch) - min(NSW_lunch)
width2 = rang2/intervals
bins2=[round(min(NSW_lunch))+round(width2*i) for i in range(0,intervals)]    

n, bins, patches = ax2.hist(NSW_lunch, bins2)
ax2.set_xlabel('Energy Output (Wh)')
ax2.set_title('NSW 12:30')

n_obs3=len(NSW_arvo)
rang3 = max(NSW_arvo) - min(NSW_arvo)
width3 = rang3/intervals
bins3=[round(min(NSW_arvo))+round(width3*i) for i in range(0,intervals)]    

n, bins, patches = ax3.hist(NSW_arvo, bins3)
ax3.set_xlabel('Energy Output (Wh)')
ax3.set_title('NSW 16:30')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize = (10,4))
intervals = 20 #n_bins
ax1.set_xlim(0,1700)
ax2.set_xlim(0,1700)
ax3.set_xlim(0,1700)
n_obs1=len(WA_morn)
rang1 = max(WA_morn) - min(WA_morn)
width1 = rang1/intervals
bins1=[round(min(WA_morn))+round(width1*i) for i in range(0,intervals)]    

n, bins, patches = ax1.hist(WA_morn, bins1)
ax1.set_xlabel('Energy Output (Wh)')
ax1.set_ylabel('Count')
ax1.set_title('WA 7:30')

n_obs2=len(WA_lunch)
rang2 = max(WA_lunch) - min(WA_lunch)
width2 = rang2/intervals
bins2=[round(min(WA_lunch))+round(width2*i) for i in range(0,intervals)]    

n, bins, patches = ax2.hist(WA_lunch, bins2)
ax2.set_xlabel('Energy Output (Wh)')
ax2.set_title('WA 12:30')

n_obs3=len(WA_arvo)
rang3 = max(WA_arvo) - min(WA_arvo)
width3 = rang3/intervals
bins3=[round(min(WA_arvo))+round(width3*i) for i in range(0,intervals)]    

n, bins, patches = ax3.hist(WA_arvo, bins3)
ax3.set_xlabel('Energy Output (Wh)')
ax3.set_title('WA 16:30')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize = (10,4))

n_obs1=len(NSW_morn)
intervals1 = 20
rang1 = max(NSW_morn) - min(NSW_morn)
width1 = rang1/intervals1 
bins1=[round(min(NSW_morn))+round(width1*i) for i in range(0,intervals1)]    

n, bins, patches = ax1.hist(NSW_morn, bins1, orientation = "horizontal")
ax1.set_xlabel('Count')
ax1.set_ylabel('Energy Output (Wh)')
ax1.set_title('NSW 7:30')
#ax1.axhline(501)

n_obs2=len(VIC_morn)
intervals2 = 20
rang2 = max(VIC_morn) - min(VIC_morn)
width2 = rang2/intervals2
bins2=[round(min(VIC_morn))+round(width2*i) for i in range(0,intervals2)]    

n, bins, patches = ax2.hist(VIC_morn, bins2, orientation = "horizontal")
ax2.set_xlabel('Count')
ax2.set_title('VIC 7:30')

n_obs3=len(WA_morn)
intervals3 = 20
rang3 = max(WA_morn) - min(WA_morn)
width3 = rang3/intervals3
bins3=[round(min(WA_morn))+round(width3*i) for i in range(0,intervals3)]    

n, bins, patches = ax3.hist(WA_morn, bins3, orientation = "horizontal")
ax3.set_xlabel('Count')
ax3.set_title('WA 7:36')
#plt.savefig('./results/randomdates/random_morning_trip.png')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize = (10,4))

n_obs1=len(NSW_lunch)
intervals1 = 20
rang1 = max(NSW_lunch) - min(NSW_lunch)
width1 = rang1/intervals1 
bins1=[round(min(NSW_lunch))+round(width1*i) for i in range(0,intervals1)]    

n, bins, patches = ax1.hist(NSW_lunch, bins1, orientation = "horizontal")
ax1.set_xlabel('Count')
ax1.set_ylabel('Energy Output (Wh)')
ax1.set_title('NSW 12:30pm')

n_obs2=len(VIC_lunch)
intervals2 = 20
rang2 = max(VIC_lunch) - min(VIC_lunch)
width2 = rang2/intervals2
bins2=[round(min(VIC_lunch))+round(width2*i) for i in range(0,intervals2)]    

n, bins, patches = ax2.hist(VIC_lunch, bins2, orientation = "horizontal")
ax2.set_xlabel('Count')
ax2.set_title('VIC 12:30pm')

n_obs3=len(WA_lunch)
intervals3 = 20
rang3 = max(WA_lunch) - min(WA_lunch)
width3 = rang3/intervals3
bins3=[round(min(WA_lunch))+round(width3*i) for i in range(0,intervals3)]    

n, bins, patches = ax3.hist(WA_lunch, bins3, orientation = "horizontal")
ax3.set_xlabel('Count')
ax3.set_title('WA 12:36pm')
#plt.savefig('./results/randomdates/random_lunch_trip.png')

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize = (10,4))

n_obs1=len(NSW_arvo)
intervals1 = 20
rang1 = max(NSW_arvo) - min(NSW_arvo)
width1 = rang1/intervals1 
bins1=[round(min(NSW_arvo))+round(width1*i) for i in range(0,intervals1)]    

n, bins, patches = ax1.hist(NSW_arvo, bins1, orientation = "horizontal")
ax1.set_xlabel('Count')
ax1.set_ylabel('Energy Output (Wh)')
ax1.set_title('NSW 16:30')

n_obs2=len(VIC_arvo)
intervals2 = 20
rang2 = max(VIC_arvo) - min(VIC_arvo)
width2 = rang2/intervals2
bins2=[round(min(VIC_arvo))+round(width2*i) for i in range(0,intervals2)]    

n, bins, patches = ax2.hist(VIC_arvo, bins2, orientation = "horizontal")
ax2.set_xlabel('Count')
ax2.set_title('VIC 16:30')

n_obs3=len(WA_arvo)
intervals3 = 20
rang3 = max(WA_arvo) - min(WA_arvo)
width3 = rang3/intervals3
bins3=[round(min(WA_arvo))+round(width3*i) for i in range(0,intervals3)]    

n, bins, patches = ax3.hist(WA_arvo, bins3, orientation = "horizontal")
ax3.set_xlabel('Count')
ax3.set_title('WA 16:36')
#plt.savefig('./results/randomdates/random_evening_trip.png')